# Colour viewer for HOYS lightcurves

* ***Shift + Enter on a code cell to run it and advance to the next cell.***

* ***If changing sliders, rerun the code block below the sliders***

* ***Plot windows are interactive, can use mouse to select and zoom into ranges***

In [1]:
#load required modules
import os
import pandas as pd
import astropy
import numpy
import math
from astropy.time import Time
from astropy.timeseries import LombScargle
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "presentation"
from plotly.offline import iplot
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output

### List available lightcurve data and select from dropdown list
* Upload your .csv file to '/light_curve_csv_files'

In [2]:
#read HOYS csv file
lc_folder = "./light_curve_csv_files/"
star_list=os.listdir(lc_folder)

lc_select = widgets.Dropdown(
    options=sorted(star_list),
    description='Select a light curve file:',
    style = {'description_width': 'initial'}
)
display(lc_select)

Dropdown(description='Select a light curve file:', options=('.DS_Store', 'lightcurve_TOri.txt', 'lightcurve_V2…

### Display head and tail of csv file containing all of the data

In [15]:
print('selected datafile:',lc_select.value)
lc_data = pd.read_csv(os.path.join(lc_folder,lc_select.value),comment='#',delimiter=' ')
pd.set_option('display.max_columns', None)
#view file head and tail
lc_data

selected datafile: lightcurve_TOri.txt


,id,calibrated_magnitude,calibrated_error,magnitude_rms_error,x,y,alpha_j2000,delta_j2000,fwhm_world,flags,magnitude,observation_id,filter,original_filter,date,user_id,device_id,target,fits_id
0,148205197,10.4579,0.028127,0.0056,444.946503,664.540405,83.959338,-5.476287,0.001537,0.0,13.4111,18017,B,B,2.458024e+06,84,131,[3] M42,16948
1,153292054,10.3159,0.039358,0.0273,714.025391,540.521973,83.959338,-5.476611,0.006464,3.0,13.1297,21135,B,B,2.456935e+06,84,131,[3] M42,19717
2,145995746,11.3558,0.030576,0.0289,827.697083,654.667603,83.959339,-5.476383,0.003866,3.0,13.7005,16249,B,B,2.457449e+06,84,131,[3] M42,15640
3,146006453,10.9175,0.039121,0.0080,902.415405,726.595886,83.959339,-5.476306,0.003452,2.0,13.3877,16264,B,B,2.457465e+06,84,131,[3] M42,15674
4,174070944,10.9172,0.037151,0.0148,790.235779,590.739075,83.959341,-5.476141,0.001691,0.0,14.2791,29354,V,V,2.458928e+06,84,131,[3] M42,28609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12268,150335939,10.3990,0.011343,0.0035,685.392517,682.193176,83.961001,-5.476103,0.001295,2.0,12.6706,19558,B,B,2.458217e+06,84,131,[3] M42,18058
12269,360043562,10.5994,0.029153,0.0124,632.510498,306.999115,83.961001,-5.476749,0.002825,0.0,13.9279,61965,I,I,2.459565e+06,84,131,[3] M42,60109
12270,360046473,10.0410,0.041848,0.0067,644.475403,277.919098,83.961002,-5.476616,0.002559,0.0,13.4080,61981,R,R,2.459553e+06,84,131,[3] M42,60123
12271,354942476,10.5125,0.048388,0.0083,640.415710,284.809509,83.961008,-5.476800,0.002174,0.0,13.7216,60124,I,I,2.459542e+06,84,131,[3] M42,58555


In [19]:
cal_error_sel=widgets.FloatSlider(
    value=0.1,
    min=0.05,
    max=0.2,
    step=0.01,
    description='Calibrated error [mag]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(cal_error_sel)

med_window_sel=widgets.FloatSlider(
    value=1,
    min=0.5,
    max=10,
    step=0.1,
    description='Median filter window [days]:',
    layout={'width': '600px'},
    style = {'description_width': 'initial'}
)
display(med_window_sel)

FloatSlider(value=0.1, description='Calibrated error [mag]:', layout=Layout(width='600px'), max=0.2, min=0.05,…

FloatSlider(value=1.0, description='Median filter window [days]:', layout=Layout(width='600px'), max=10.0, min…

In [20]:
band=['B','V','R','I']
cal_error=cal_error_sel.value
med_window=med_window_sel.value

filtered_data=lc_data[(lc_data['filter'].isin(band)) & (lc_data['flags'] <= 4) & (lc_data['calibrated_error'] < cal_error) & (lc_data['calibrated_magnitude'] > 0.0) & (lc_data['calibrated_error'] > 0.0) & (lc_data['fwhm_world'] > 0.0) & (lc_data['fwhm_world'] < 9.0/3600.0)]
filtered_data=filtered_data.copy()
filtered_data['mjd']=Time(filtered_data.date,format='jd').mjd
filtered_data.reset_index(inplace=True,drop=True)
#filtered_data['utc']=pd.to_datetime(filtered_data['date'],unit='D', origin='julian')

#start=Time.now()

# slow method in pandas
# for date in filtered_data.index:
#     med_mask=(abs(filtered_data['date'][date] - filtered_data['date']) < 1)
#     for fil in band:
#         filtered_data.loc[med_mask,fil]=numpy.median(filtered_data[med_mask]['calibrated_magnitude'][(filtered_data[med_mask]['filter']==fil)])

#fast method with numpy arrays
date=numpy.array(filtered_data['date'])
filt=numpy.array(filtered_data['filter'])
mag=numpy.array(filtered_data['calibrated_magnitude'])
dt2=numpy.dtype([('B',numpy.float32),('V',numpy.float32),('R',numpy.float32),('I',numpy.float32),
                ('Be',numpy.float32),('Ve',numpy.float32),('Re',numpy.float32),('Ie',numpy.float32)])
colours=numpy.empty(mag.shape,dtype=dt2)
colours[:]=numpy.nan
for i in range(0,len(filtered_data)):
    for f in band:
        check = numpy.where( (filt == f) &  (numpy.abs(date[i] - date) < med_window) )
        if len(check[0]>0):
            colours[i][f]=numpy.median(mag[check[0]])
            colours[i][f+'e'] = numpy.nanstd(mag[check[0]])/numpy.sqrt(float(len(mag[check[0]])))

        
filtered_data=pd.concat((filtered_data,pd.DataFrame(colours)),axis=1)

display(filtered_data.sort_values(by='date'))

#end=Time.now()
#print((end-start).sec)

,id,calibrated_magnitude,calibrated_error,magnitude_rms_error,x,y,alpha_j2000,delta_j2000,fwhm_world,flags,magnitude,observation_id,filter,original_filter,date,user_id,device_id,target,fits_id,mjd,B,V,R,I,Be,Ve,Re,Ie
9110,159319172,9.8033,0.073086,0.0030,695.592407,655.536926,83.960470,-5.476142,0.001344,3.0,11.5369,24756,V,V,2.456536e+06,84,131,[3] M42,22052,56535.409363,NaN,9.9949,NaN,NaN,NaN,0.038024,NaN,NaN
8520,159317849,9.9560,0.021891,0.0027,695.191223,655.722778,83.960390,-5.476208,0.001543,3.0,11.5726,24758,V,V,2.456536e+06,84,131,[3] M42,22053,56535.409549,NaN,9.9949,NaN,NaN,NaN,0.038024,NaN,NaN
8605,159318489,9.9949,0.026299,0.0026,705.933411,649.268677,83.960399,-5.476259,0.001228,3.0,11.6412,24757,V,V,2.456537e+06,84,131,[3] M42,22054,56536.408449,NaN,9.9649,NaN,NaN,NaN,0.028916,NaN,NaN
9158,159317173,10.0316,0.027255,0.0022,705.792480,649.612915,83.960479,-5.476115,0.001438,3.0,11.6747,24759,V,V,2.456537e+06,84,131,[3] M42,22055,56536.408623,NaN,9.9649,NaN,NaN,NaN,0.028916,NaN,NaN
8813,159311674,10.0328,0.034928,0.0023,705.794678,649.661682,83.960422,-5.476164,0.001804,3.0,11.6680,24761,V,V,2.456537e+06,84,131,[3] M42,22057,56536.408808,NaN,9.9649,NaN,NaN,NaN,0.028916,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051,371113549,10.3139,0.045016,0.0106,939.968628,392.114899,83.960056,-5.476566,0.001957,3.0,13.7544,64697,B,B,2.459635e+06,71,204,[3] M42,63676,59634.839167,10.3139,9.8066,9.7667,NaN,0.0,0.000000,0.0,NaN
1417,371102216,9.7667,0.033737,0.0080,938.690491,390.638306,83.959860,-5.476717,0.001417,3.0,12.2099,64695,R,R,2.459635e+06,71,204,[3] M42,63674,59634.839919,10.3139,9.8066,9.7667,NaN,0.0,0.000000,0.0,NaN
2495,371141742,9.7352,0.023799,0.0048,953.406677,370.109894,83.960009,-5.476551,0.001427,3.0,12.3549,64787,V,V,2.459639e+06,71,204,[3] M42,63717,59638.786215,9.9472,9.7352,9.8177,NaN,0.0,0.000000,0.0,NaN
3496,371142209,9.9472,0.033740,0.0146,952.682678,369.991913,83.960086,-5.476681,0.001607,3.0,13.7313,64790,B,B,2.459639e+06,71,204,[3] M42,63720,59638.787315,9.9472,9.7352,9.8177,NaN,0.0,0.000000,0.0,NaN


### Lightcurve plot

In [47]:
def plot_lightcurve(date_range_sel=[min(filtered_data['mjd']),max(filtered_data['mjd'])],error_bars=False):
    global filtered_data
    filtered_data=filtered_data[(filtered_data['mjd'] > date_range_sel[0]) & (filtered_data['mjd'] < date_range_sel[1])]

    color_discrete_map = {'B': 'blue', 'V': 'green', 'R': 'red', 'I':'black'}

    #plot lightcurve
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'},
                       error_y=filtered_data.calibrated_error)
    else:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data.calibrated_magnitude,color='filter',color_discrete_map=color_discrete_map,labels={'x':'Date [mjd]'})
    fig1['layout']['yaxis_title']='Magnitude [mag]'
    fig1['layout']['xaxis_title']=f'Date [mjd]'
    fig1['layout']['yaxis']['autorange'] = "reversed"
    fig1['layout']['xaxis']['tickformat'] = "3f"

    iplot(fig1)
    
interact(plot_lightcurve,date_range_sel=widgets.FloatRangeSlider(value=[min(filtered_data['mjd']),max(filtered_data['mjd'])],min=min(filtered_data['mjd']),max=max(filtered_data['mjd']),step=0.1,
    description='Date range [mjd]:',readout_format='.1f',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False))





interactive(children=(FloatRangeSlider(value=(56536.40844907006, 59638.78621528018), continuous_update=False, …

<function __main__.plot_lightcurve(date_range_sel=[56536.40844907006, 59638.78621528018], error_bars=False)>

### Colour curve plot

In [46]:
def view_colourcurve(col1='V',col2='I',error_bars=False):
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'},
                    error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])))
    else:
        fig1=px.scatter(filtered_data,x=filtered_data.mjd,y=filtered_data[col1]-filtered_data[col2],labels={'x':'Date [mjd]', 'y':f'{col1}-{col2} [mag]'})
    fig1['layout']['xaxis']['tickformat'] = "3f"
    fig1['layout']['xaxis_title']=f'Date [mjd]'

    iplot(fig1)

interact(view_colourcurve,col1= widgets.Dropdown(options=(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=(band),value='I',description='Select second colour',style = {'description_width': 'initial'}))


interactive(children=(Dropdown(description='Select first colour', index=1, options=('B', 'V', 'R', 'I'), style…

<function __main__.view_colourcurve(col1='V', col2='I', error_bars=False)>

### Colour-magnitude plot

In [45]:
def plot_cmd(m1='V',col1='V',col2='I',A_v_l=1,A_v_c=numpy.min(filtered_data['V']-filtered_data['I']),A_v_m=numpy.min(filtered_data['V']),error_bars=False):
    jd=Time(filtered_data.date,format='jd') 
    mjd=jd.mjd #plot mjd instead of jd
    if error_bars==True:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd,
                   error_y=numpy.sqrt(numpy.square(filtered_data[col1+'e']) + numpy.square(filtered_data[col2+'e'])),error_x=filtered_data[m1+'e'],
                       labels={'x':f'Colour ({col1}-{col2})', 'y':f'Magnitude ({m1}) ','color':'date'})
    else:
        fig1=px.scatter(filtered_data,x=filtered_data[col1]-filtered_data[col2],y=filtered_data[m1],color=mjd,
                        labels={'Y':f'Magnitude ({m1})','x':f'Colour ({col1}-{col2})', 'color':'date'})

    A_3p1 = {'B': 4.7, 'V': 3.55, 'R': 2.66, 'I':1.7}
    
    A_mag_3p1=A_v_l * A_3p1[m1]/A_3p1['V']
    A_col_3p1=A_v_l * ( (A_3p1[col1]/A_3p1['V']) - (A_3p1[col2]/A_3p1['V']) )
        
    xi = A_v_c
    yi = A_v_m
    fig2=px.line(x=(xi,xi+A_col_3p1),y=(yi,yi+A_mag_3p1),labels='R_v=3.1')
    fig2.update_traces(line=dict(color = 'blue',width=3),name='R_v=3.1')

    A_5 = {'B': 3.67, 'V': 3.06, 'R': 2.43, 'I':1.7}
    
    A_mag_5=A_v_l * A_5[m1]/A_5['V']
    A_col_5=A_v_l * ( (A_5[col1]/A_5['V']) - (A_5[col2]/A_5['V']) )
        
    xi = A_v_c
    yi = A_v_m
    fig3=px.line(x=(xi,xi+A_col_5),y=(yi,yi+A_mag_5),labels='R_v=5.0')
    fig3.update_traces(line=dict(color = 'green',width=3),name='R_v=5.0')
   
    fig4 = go.Figure(data=fig2.data  + fig3.data + fig1.data)
    fig4['layout']['yaxis_title']=f'Magnitude ({m1}) [mag]'
    fig4['layout']['xaxis_title']=f'Colour ({col1}-{col2}) [mag]'
    fig4['layout']['yaxis']['autorange'] = "reversed"
    fig4['layout']['coloraxis_colorbar']['title'] = 'Date [mjd]'
    fig4['layout']['coloraxis']['colorscale'] = 'Matter'
    fig4['data'][0]['showlegend']=True
    fig4['data'][1]['showlegend']=True
    fig4.update_layout(legend=dict(yanchor="top",xanchor="right"))
    fig4.update_layout(autosize=False,width=900,height=600)
    fig4.update_layout(legend_title="Extinction Vector")
    
    fig4.show()
    #iplot(fig1) 
    
interact(plot_cmd,m1= widgets.Dropdown(options=(band),value='V',description='Select magnitude',style = {'description_width': 'initial'}),
                         col1= widgets.Dropdown(options=(band),value='V',description='Select first colour',style = {'description_width': 'initial'}),
                          col2= widgets.Dropdown(options=(band),value='I',description='Select second colour',style = {'description_width': 'initial'}),
                        A_v_l=widgets.FloatSlider(value=1,min=1,max=5,step=0.01,description='Extinction vector start length [mag]:',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False),
                        A_v_c=widgets.FloatSlider(value=numpy.min(filtered_data['V']-filtered_data['I']),min=numpy.min(filtered_data['V']-filtered_data['I'])-1,max=numpy.min(filtered_data['V']-filtered_data['I'])+1,step=0.01,description='Extinction vector start colour [mag]:',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False),
                        A_v_m=widgets.FloatSlider(value=numpy.min(filtered_data['V']),min=numpy.min(filtered_data['V'])-2,max=numpy.min(filtered_data['V'])+2,step=0.01,description='Extinction vector start magnitude [mag]:',layout={'width': '600px'},style = {'description_width': 'initial'},continuous_update=False)
        )
   

interactive(children=(Dropdown(description='Select magnitude', index=1, options=('B', 'V', 'R', 'I'), style=De…

<function __main__.plot_cmd(m1='V', col1='V', col2='I', A_v_l=1, A_v_c=-1.0645504, A_v_m=9.4159, error_bars=False)>